In [1]:
import os
import timm
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from transformers import AutoModel
import torchvision.transforms as T
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:

file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_measurements.csv"

df = pd.read_csv(file_path, sep="\t")

df = df.sample(n=1000, random_state=99)



In [3]:
df.head(2)

,ImageFilePath,Genus,ScientificName,Length,Width
1921,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus,Synuchus impunctatus,0.603504,0.270993
4058,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Pterostichus,Pterostichus lachrymosus,0.833583,0.423888


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModel.from_pretrained("facebook/dinov2-base").to(device)


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
def extract_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model(image_tensor).last_hidden_state.mean(dim=1)

    
    return features.cpu().numpy()
    

In [6]:
df['Embedding'] = [extract_features(img) for img in tqdm(df['ImageFilePath'], desc='Extracting Embeddings')]

X = np.vstack(df['Embedding'].values)
X = np.hstack((X, df[['Length', 'Width']].values))

label_encoder = LabelEncoder()

y = label_encoder.fit_transform(df['ScientificName'])


Extracting Embeddings: 100%|██████████| 1000/1000 [01:22<00:00, 12.09it/s]


In [7]:
df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [15]:
seed = 42

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),   
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T

/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


NaiveBayes                | Acc: 54.00% | Prec: 51.04% | Rec: 54.00% | F1: 50.00% | Bal Acc: 29.10% | MCC: 0.5082


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precisio

LogisticRegression        | Acc: 76.00% | Prec: 73.12% | Rec: 76.00% | F1: 72.90% | Bal Acc: 44.71% | MCC: 0.7449
NearestNeighbor           | Acc: 53.50% | Prec: 49.18% | Rec: 53.50% | F1: 49.09% | Bal Acc: 32.03% | MCC: 0.5020
MLP_Baseline              | Acc: 72.00% | Prec: 70.05% | Rec: 72.00% | F1: 69.31% | Bal Acc: 42.56% | MCC: 0.7025


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [16]:
test_df = test_df.assign(**{f"Pred_{name}": label_encoder.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,ScientificName,Length,Width,Embedding,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
116,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Pterostichus,Pterostichus melanarius,0.978987,0.531729,"[[-0.6384573, -0.4578342, 0.7260156, 0.1524632...",Pterostichus melanarius,Pterostichus melanarius,Pterostichus melanarius,Pterostichus melanarius
6955,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Euryderus,Euryderus grossus,0.870635,0.600092,"[[-0.6076648, -1.0757282, 1.3042265, 0.5767869...",Euryderus grossus,Euryderus grossus,Euryderus grossus,Euryderus grossus


In [17]:
test_df.to_csv("/users/PAS2136/rayees/CV4A - MultiModality/Subset/DINOv2-traits.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/CV4A - MultiModality/Subset/DINOv2-traits-metrics.csv", index=False)